In [1]:
import numpy as np
import scipy as sc
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.io.wavfile as wavfile
from sklearn import preprocessing
import os
from skimage.transform import resize
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold, GridSearchCV
from scipy import stats
from sklearn import neural_network
import seaborn as sns
import csv
import pandas as pd
import itertools

## Notes on data collection
Data was collected using my Xiaomi Mi 9, through the Phyphox app. The phone was housed in a armband case, tied to my right arm, as shown in the videos.

### Data Loading and Preprocessing
Data (organized in separate folders), was loaded and prepocessed through a loop, which opened CSV files per folder, saved the data (Accelerometer x,y,z, time and gyroscope x,y,z) to a list. The first 10 seconds of each recordings are omitted, due to the artifacts created by the activation of the recording app. For the same reason, the last 10 seconds of each recording are also omitted. 

- Walking data was collected in 14 10-minute sessions
- Squat data was recorder in 10 squats/session
- Meditation was used as additional activity, recorded in 15 mins session
- Stair walking was recorded in short 2 min intervals.

Data structure: HW4_Data/NameOfClass/Class1/Accelerometer.csv

Data was collected into dataframes, one per activity, labelled.
The dataframes were collected in df_list

In [2]:
time = []
labels = []
acc_x = []
acc_y = []
acc_z = []
gyro_x = []
gyro_y = []
gyro_z = []
df_list = []
print(os.path)
for folder in os.listdir('./HW4_data'):
    labels = []
    acc_x = []
    acc_y = []
    acc_z = []
    gyro_x = []
    gyro_y = []
    gyro_z = []
    if folder == 'sensors.txt':
        pass
    else:
        path=os.path.join('./HW4_data',folder)
        print(path)
        filecount = 0
        for subfolder in os.listdir(path):
            path_subfolder=os.path.join(path, subfolder)
            print(path_subfolder)
            curr_acc_file =os.path.join(path_subfolder, 'Accelerometer.csv')
            curr_gyro_file = os.path.join(path_subfolder, 'Gyroscope.csv')
            tot_rows = 0
            with open(curr_gyro_file) as csv_file1:
                csv_read = csv.reader(csv_file1, delimiter=',')
                tot_rows =  sum(1 for line in csv_read)
            with open(curr_acc_file) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                #First/ Last 10 seconds at 0.01s recordings = 1000 rows
                start_time = int(5 // 0.01)
                end_time = tot_rows - int(5 // 0.01)
                line_count = 0
                print(csv_reader)
                for row in csv_reader:
                    if line_count == 0:
                        print(f'Column names are {", ".join(row)}')
                        line_count  = 1 + start_time
                    elif line_count <= end_time:
                        #0 is time, so ignore it
                        acc_x.append(row[1])
                        acc_y.append(row[2])
                        acc_z.append(row[3])
                        line_count += 1
                    if line_count > end_time:
                        print("done with ", subfolder, "acc, at line", line_count)
                        #faster by breaking out of loop when not needed
                        break
                    labels.append(folder)
            #Overwrite filename to save memory
            with open(curr_gyro_file) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                #First/ Last 10 minutes at 0.01s recordings = 1000 rows
                start_time = int(5 // 0.01) 
                end_time = tot_rows - int(5 // 0.01) 
                line_count = 0
                for row in csv_reader:
                    if line_count == 0:
                        print(f'Column names are {", ".join(row)}')
                        line_count  = 1 + start_time
                    elif line_count <= end_time:
                        #0 is time, so ignore it
                        gyro_x.append(row[1])
                        gyro_y.append(row[2])
                        gyro_z.append(row[3])
                        line_count += 1
                    if line_count > end_time:
                        print("done with ", subfolder, "gyro, at line ", line_count)
                        #faster by breaking out of loop when not needed
                        break
        df = pd.DataFrame({
        'label': labels,
        'acc_x': acc_x,
        'acc_y': acc_y,
        'acc_z': acc_z,
        'gyro_x': gyro_x,
        'gyro_y': gyro_y,
        'gyro_z': gyro_z
        })
    df_list.append(df)


<module 'ntpath' from 'C:\\Users\\Irene\\Miniconda3\\lib\\ntpath.py'>
./HW4_data\Climbing_down
./HW4_data\Climbing_down\down1
Column names are Time (s), X (m/s^2), Y (m/s^2), Z (m/s^2)
done with  down1 acc, at line 3352
Column names are Time (s), X (rad/s), Y (rad/s), Z (rad/s)
done with  down1 gyro, at line  3352
./HW4_data\Climbing_down\down10
Column names are Time (s), X (m/s^2), Y (m/s^2), Z (m/s^2)
done with  down10 acc, at line 3376
Column names are Time (s), X (rad/s), Y (rad/s), Z (rad/s)
done with  down10 gyro, at line  3376
./HW4_data\Climbing_down\down2
Column names are Time (s), X (m/s^2), Y (m/s^2), Z (m/s^2)
done with  down2 acc, at line 2727
Column names are Time (s), X (rad/s), Y (rad/s), Z (rad/s)
done with  down2 gyro, at line  2727
./HW4_data\Climbing_down\down3
Column names are Time (s), X (m/s^2), Y (m/s^2), Z (m/s^2)
done with  down3 acc, at line 3415
Column names are Time (s), X (rad/s), Y (rad/s), Z (rad/s)
done with  down3 gyro, at line  3415
./HW4_data\Climbin

done with  squats4 gyro, at line  21783
./HW4_data\Squats\squats5
Column names are Time (s), Acceleration x (m/s^2), Acceleration y (m/s^2), Acceleration z (m/s^2)
done with  squats5 acc, at line 22590
Column names are Time (s), Gyroscope x (rad/s), Gyroscope y (rad/s), Gyroscope z (rad/s)
done with  squats5 gyro, at line  22590
./HW4_data\Squats\squats6
Column names are Time (s), Acceleration x (m/s^2), Acceleration y (m/s^2), Acceleration z (m/s^2)
done with  squats6 acc, at line 20431
Column names are Time (s), Gyroscope x (rad/s), Gyroscope y (rad/s), Gyroscope z (rad/s)
done with  squats6 gyro, at line  20431
./HW4_data\Walking
./HW4_data\Walking\walking1
Column names are Time (s), X (m/s^2), Y (m/s^2), Z (m/s^2)
done with  walking1 acc, at line 62279
Column names are Time (s), X (rad/s), Y (rad/s), Z (rad/s)
done with  walking1 gyro, at line  62279
./HW4_data\Walking\walking10
Column names are Time (s), Acceleration x (m/s^2), Acceleration y (m/s^2), Acceleration z (m/s^2)
done w

Feature Scaling using MIN-MAX Scaler (range 0 to 1)

In [3]:
min_max_scaler = preprocessing.MinMaxScaler() 
for dataframe in df_list:
    dataframe[["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z" ]] = min_max_scaler.fit_transform(dataframe[["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z" ]])

Features considered, for each of the direction (X,Y,Z) of both Accelerometer and Gyroscope, were:
- Max 
- Min 
- Average 
- Standard deviation
- Variance 

The frame of the window was 2000 rows, or 20 seconds. Each feature was considered for the previous 20 seconds
  

In [ ]:
#N sets the window size
N = 20
max_freq_arr_x = []
min_freq_arr_x = []
mean_freq_arr_x = []
std_freq_arr_x = []
var_freq_arr_x = []
mode_freq_arr_x = []

max_freq_arr_y = []
min_freq_arr_y = []
mean_freq_arr_y = []
std_freq_arr_y = []
var_freq_arr_y = []
mode_freq_arr_y = []

max_freq_arr_z = []
min_freq_arr_z = []
mean_freq_arr_z = []
std_freq_arr_z = []
var_freq_arr_z = []
mode_freq_arr_z = []

max_freq_q_x = []
min_freq_q_x = []
mean_freq_q_x = []
std_freq_q_x = []
var_freq_q_x = []
mode_freq_q_x = []

max_freq_q_y = []
min_freq_q_y = []
mean_freq_q_y = []
std_freq_q_y = []
var_freq_q_y = []
mode_freq_q_y = []

max_freq_q_z = []
min_freq_q_z = []
mean_freq_q_z = []
std_freq_q_z = []
var_freq_q_z = []
mode_freq_q_z = []
labels = []

for dataframe in df_list:
    for i in range(0, len(dataframe)):
        if i % N ==0 and i != 0:
            labels.append(dataframe.iloc[0,0])
            max_freq_arr_x.append(np.amax(dataframe.iloc[i-N:i,1]))
            min_freq_arr_x.append(np.amin(dataframe.iloc[i-N:i,1]))
            mean_freq_arr_x.append(np.mean(dataframe.iloc[i-N:i,1]))
            std_freq_arr_x.append(np.std(dataframe.iloc[i-N:i,1]))
            var_freq_arr_x.append(np.var(dataframe.iloc[i-N:i,1]))
#             mode_freq_arr_x.append(stats.mode(dataframe.iloc[i-2000:i,1]))[0]
            max_freq_arr_y.append(np.amax(dataframe.iloc[i-N:i,2]))
            min_freq_arr_y.append(np.amin(dataframe.iloc[i-N:i,2]))
            mean_freq_arr_y.append(np.mean(dataframe.iloc[i-N:i,2]))
            std_freq_arr_y.append(np.std(dataframe.iloc[i-N:i,2]))
            var_freq_arr_y.append(np.var(dataframe.iloc[i-N:i,2]))
            
            max_freq_arr_z.append(np.amax(dataframe.iloc[i-N:i,3]))
            min_freq_arr_z.append(np.amin(dataframe.iloc[i-N:i,3]))
            mean_freq_arr_z.append(np.mean(dataframe.iloc[i-N:i,3]))
            std_freq_arr_z.append(np.std(dataframe.iloc[i-N:i,3]))
            var_freq_arr_z.append(np.var(dataframe.iloc[i-N:i,3]))
            
            max_freq_q_x.append(np.amax(dataframe.iloc[i-N:i,4]))
            min_freq_q_x.append(np.amin(dataframe.iloc[i-N:i,4]))
            mean_freq_q_x.append(np.mean(dataframe.iloc[i-N:i,4]))
            std_freq_q_x.append(np.std(dataframe.iloc[i-N:i,4]))
            var_freq_q_x.append(np.var(dataframe.iloc[i-N:i,4]))
            
            max_freq_q_y.append(np.amax(dataframe.iloc[i-N:i,5]))
            min_freq_q_y.append(np.amin(dataframe.iloc[i-N:i,5]))
            mean_freq_q_y.append(np.mean(dataframe.iloc[i-N:i,5]))
            std_freq_q_y.append(np.std(dataframe.iloc[i-N:i,5]))
            var_freq_q_y.append(np.var(dataframe.iloc[i-N:i,5]))
            
            max_freq_q_z.append(np.amax(dataframe.iloc[i-N:i,6]))
            min_freq_q_z.append(np.amin(dataframe.iloc[i-N:i,6]))
            mean_freq_q_z.append(np.mean(dataframe.iloc[i-N:i,6]))
            std_freq_q_z.append(np.std(dataframe.iloc[i-N:i,6]))
            var_freq_q_z.append(np.var(dataframe.iloc[i-N:i,6]))
            


In [ ]:
#concatenating all features in one df
large_df = pd.DataFrame(
    {'labels': labels,
     'max_freq_arr_x': max_freq_arr_x,
     'min_freq_arr_x': min_freq_arr_x,
     'mean_freq_arr_x': mean_freq_arr_x,
     'std_freq_arr_x': std_freq_arr_x,
     'var_freq_arr_x': var_freq_arr_x,
     
     'max_freq_arr_y': max_freq_arr_y,
     'min_freq_arr_y': min_freq_arr_y,
     'mean_freq_arr_y': mean_freq_arr_y,
     'std_freq_arr_y': std_freq_arr_y,
     'var_freq_arr_y': var_freq_arr_y,
     
     'max_freq_arr_z': max_freq_arr_z,
     'min_freq_arr_z': min_freq_arr_z,
     'mean_freq_arr_z': mean_freq_arr_z,
     'std_freq_arr_z': std_freq_arr_z,
     'var_freq_arr_z': var_freq_arr_z,
     
     'max_freq_q_x': max_freq_q_x,
     'min_freq_q_x': min_freq_q_x,
     'mean_freq_q_x': mean_freq_q_x,
     'std_freq_q_x': std_freq_q_x,
     'var_freq_q_x': var_freq_q_x, 
     
     'max_freq_q_y': max_freq_q_y,
     'min_freq_q_y': min_freq_q_y,
     'mean_freq_q_y': mean_freq_q_y,
     'std_freq_q_y': std_freq_q_y,
     'var_freq_q_y': var_freq_q_y, 
     
     'max_freq_q_z': max_freq_q_z,
     'min_freq_q_z': min_freq_q_z,
     'mean_freq_q_z': mean_freq_q_z,
     'std_freq_q_z': std_freq_q_z,
     'var_freq_q_z': var_freq_q_z })



# #Concatenated df into one
# print(df_list[0:3])
# df = pd.concat(df_list)
# print(df.head())
# print(var_freq_arr_x)

In [ ]:
df_balance_test = large_df.groupby(['labels'])['labels'].count()
print(df_balance_test)
# plt.fig()
df_balance_test.plot(kind = 'bar')
# plt.fig()
# plt.title('No of Datapoints per Activity', fontsize=15)
# sns.countplot(df_balance_test)
# plt.xticks(rotation=90)
# plt.show()

Walking and meditation represent the largest casses. This could be due to the fact that a larger window size was chosen, which could lead to excluding part of the stairs data that was collected in short segments. 

In [6]:
laben = preprocessing.LabelEncoder()
dx_en = laben.fit_transform(large_df["labels"])
large_df['labels'] = dx_en
#Data selection
x_bin = large_df.iloc[:,1:].values
y_bin = large_df.iloc[:,0:1].values
print(y_bin[:10])
print(x_bin[:10])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[0.6655253  0.17004412 0.35117601 0.06738314 0.00454049 0.93505108
  0.44706199 0.62764363 0.08074449 0.00651967 0.64677766 0.09340949
  0.28729063 0.10050998 0.01010226 0.8450621  0.         0.58691204
  0.10109766 0.01022074 0.81178191 0.30527425 0.49642019 0.06849088
  0.004691   0.76162236 0.35232265 0.58082037 0.08186089 0.0067012 ]
 [0.96177526 0.12871629 0.3751188  0.08421639 0.0070924  0.85379998
  0.16407026 0.53268758 0.11025686 0.01215658 1.         0.00469208
  0.31695787 0.15361572 0.02359779 0.91111017 0.01460651 0.57618861
  0.11429391 0.0130631  0.94548198 0.10930666 0.49433956 0.09534089
  0.00908989 0.91452812 0.03020042 0.56413118 0.11806705 0.01393983]
 [0.74590096 0.08026866 0.44989122 0.10258639 0.01052397 0.89996938
  0.17744046 0.44220258 0.11468613 0.01315291 0.91749369 0.02664695
  0.31966555 0.1614065  0.02605206 0.99552324 0.10885838 0.57390458
  0.14267324 0.02035565 0.84850752 0.1242469  0.5050307  0.1041

## Methods for running models
Useful for standard sklearn models

In [19]:
#cross - validation on 10 folds
def cross_validate(x_bin, y_bin, classifier):
    scores = []
    cv = KFold(n_splits=10, random_state=42, shuffle=True)
    for train_index, test_index in cv.split(x_bin):
            X_train1, X_test1, y_train1, y_test1 = x_bin[train_index], x_bin[test_index], y_bin[train_index], y_bin[test_index]
            classifier.fit(X_train1, y_train1)
            scores.append(classifier.score(X_test1, y_test1))
    print("Mean accuracy: " + str(np.mean(scores)))
    return np.mean(scores)

#Run best model after gridsearch
def run_model(classifier, X_train, y_train, X_test, y_test):
    
    classifier = classifier.fit(X_train, y_train)
    score_class = (classifier.score(X_test, y_test))
    test_accuracy = np.mean(score_class)
    y_pred = classifier.predict(X_test)
    print("Mean accuracy: " + str(score_class))
    train_error = classifier.score(X_train, y_train)
    print("Training Score: ", train_error)
    test_error = classifier.score(X_test, y_test)
    print("Testing Score: ", test_error)
    target_names = ['Climbing_down', 'Climbing_up', 'Extra', 'Squats', 'Walking']
    print(metrics.classification_report(y_test, y_pred.round(), target_names=target_names))
    cm = metrics.confusion_matrix(y_test, y_pred)
    print(cm)
    return cm

## Logistic Regression
Gridsearch was firs performed to find the optimal parameter combination

In [8]:
#Splitting Data
from sklearn.model_selection import train_test_split
X_train1, X_vali, y_train1, y_vali = train_test_split(x_bin, y_bin, test_size = 0.20, random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size = 0.25, random_state = 0)


In [9]:
parameters = {'C':[0.01, 0.1, 1, 10, 20, 30], 'penalty':['l2','l1']}
log_reg = linear_model.LogisticRegression()
log_reg_grid = GridSearchCV(log_reg, param_grid=parameters, cv=3, verbose=1, n_jobs=-1)
log_reg_grid.fit(X_train, y_train)
print('The following are the values of the tuned hyper-parameters:')
print(log_reg_grid.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    3.8s finished
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The following are the values of the tuned hyper-parameters:
{'C': 30, 'penalty': 'l2'}


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


After finding the optimal parameters for Logistic Regression (C = 20, Penalty = L2), a model was run, and accuracies, as well as a confusion matrix, were produced.

In [10]:
log_reg = linear_model.LogisticRegression(C = 30, penalty = 'l2')
cm = run_model(log_reg, X_train, y_train, X_test, y_test)



C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean accuracy: 0.9981617647058824
Training Score:  1.0
Testing Score:  0.9981617647058824
               precision    recall  f1-score   support

Climbing_down       1.00      1.00      1.00         3
  Climbing_up       0.67      1.00      0.80         2
        Extra       1.00      1.00      1.00       286
       Squats       1.00      1.00      1.00        16
      Walking       1.00      1.00      1.00       237

     accuracy                           1.00       544
    macro avg       0.93      1.00      0.96       544
 weighted avg       1.00      1.00      1.00       544

[[  3   0   0   0   0]
 [  0   2   0   0   0]
 [  0   0 286   0   0]
 [  0   0   0  16   0]
 [  0   1   0   0 236]]


In [20]:
#cross - validation on 10 folds
print("Cross validation on new data: ")
log_reg_acc = cross_validate(X_vali, y_vali, log_reg)



Cross validation on new data: 


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Mean accuracy: 0.9963299663299663


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the num

## SVM on Binned features
Gridsearch was firs performed to find the optimal parameter combination

In [14]:
parameters = [ {
'kernel': ["linear"],
'C': [10, 100, 500],
'gamma': [0.01, 0.1, 1 ]
} ]

classifier = SVC() 
clf = GridSearchCV(estimator = classifier, param_grid = parameters, cv=10)
clf.fit(X_train, y_train)
print('The following are the values of the tuned hyper-parameters:')
print(clf.best_params_)


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = col

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

The following are the values of the tuned hyper-parameters:
{'C': 10, 'gamma': 0.01, 'kernel': 'linear'}


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
svc = SVC(kernel='linear', C = 10, gamma = 0.01)
# cross_validation is always on new data
run_model(svc, X_train, y_train, X_test, y_test)
svc_acc = cross_validate(X_vali, y_vali, svc)

Mean accuracy: 0.9981617647058824
Training Score:  1.0
Testing Score:  0.9981617647058824
               precision    recall  f1-score   support

Climbing_down       1.00      1.00      1.00         3
  Climbing_up       0.67      1.00      0.80         2
        Extra       1.00      1.00      1.00       286
       Squats       1.00      1.00      1.00        16
      Walking       1.00      1.00      1.00       237

     accuracy                           1.00       544
    macro avg       0.93      1.00      0.96       544
 weighted avg       1.00      1.00      1.00       544

[[  3   0   0   0   0]
 [  0   2   0   0   0]
 [  0   0 286   0   0]
 [  0   0   0  16   0]
 [  0   1   0   0 236]]
Mean accuracy: 0.9981818181818183


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Model 3: MLP
Gridsearch was performed first to determine optimal parameters

In [16]:
#Gridsearch to find the optimized parameters 
parameters = [ {
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [(64,32), (32,16), (16,8)],
'alpha': [1e-1, 1e-3, 1e-5],
'activation': ["logistic", "relu", "tanh"]
} ]


mlp = neural_network.MLPClassifier(learning_rate_init=0.0001, max_iter = 1000)
clf = GridSearchCV(estimator = mlp, param_grid = parameters, cv=10)
clf.fit(X_train, y_train)

print('The following are the values of the tuned hyper-parameters:')
print(clf.best_params_)

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached an

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

...

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached an

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached an

The following are the values of the tuned hyper-parameters:
{'activation': 'tanh', 'alpha': 1e-05, 'hidden_layer_sizes': (64, 32), 'learning_rate': 'adaptive'}


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
#Testing the best model
mlp = neural_network.MLPClassifier(hidden_layer_sizes = (64,32), activation='tanh', learning_rate_init=0.0001, alpha = 1e-05, learning_rate='adaptive', max_iter = 9000)

run_model(mlp, X_train, y_train, X_test, y_test)
mlp_acc = cross_validate(X_vali, y_vali, mlp)

C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean accuracy: 0.9797794117647058
Training Score:  0.9901960784313726
Testing Score:  0.9797794117647058
               precision    recall  f1-score   support

Climbing_down       1.00      0.33      0.50         3
  Climbing_up       0.00      0.00      0.00         2
        Extra       0.98      1.00      0.99       286
       Squats       0.83      0.62      0.71        16
      Walking       1.00      1.00      1.00       237

     accuracy                           0.98       544
    macro avg       0.76      0.59      0.64       544
 weighted avg       0.98      0.98      0.98       544

[[  1   1   1   0   0]
 [  0   0   0   2   0]
 [  0   0 286   0   0]
 [  0   0   5  10   1]
 [  0   1   0   0 236]]


C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Irene\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y 

Mean accuracy: 0.9797306397306398


IndexError: invalid index to scalar variable.

In [30]:
print('Logistic Regression Cross-Validation Accuracy : {:.04}%'.format(log_reg_acc* 100))

print('Linear SVC Cross-Validation Accuracy        : {:.04}% '.format(svc_acc * 100))

print('MLP Cross-Validation Accuracy : {:.04}%'.format(mlp_acc * 100))

Logistic Regression Cross-Validation Accuracy : 99.63%
Linear SVC Cross-Validation Accuracy        : 99.82% 
MLP Cross-Validation Accuracy : 97.97%


## Conclusions
In all 3 models tested, accuracy was above 95% for all trials. For one model (Random Forest, not shown), accuracy ws sligtly lower (~94% on all trials). High accuracy could be due to class imbalance, as well as the choice of additional activity (meditation). To balance classes, more data points should be collected on squats, as 50 squats only made up ~15/20 minutes of results, while walking was registered for two hours. For better accuracy, the data was split in testing, training and validation sets, with a 20:60:20 split. This ensured that the data used for 10-fold cross-validation had not been yet used. 